In [55]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.svm import SVR
from sklearn.linear_model import SGDRegressor
from sklearn.decomposition import PCA
from sklearn.cross_decomposition import PLSRegression
from sklearn.metrics import mean_squared_error
import os

In [56]:
exps_dir = "../../exps"
if os.path.exists(exps_dir) == False: # tạo thư mục (nếu chưa có)
  os.makedirs(exps_dir, exist_ok=True)

save_dir = f"{exps_dir}"
os.makedirs(save_dir, exist_ok=True)
test_size=0.33
seed=42

In [57]:
data= pd.read_excel('../../data/May 13.xls')
data

,id,Company,Index,Year,CashRatio,EBIT,EBITDA,GrossProfitMargin,EBITDARatio,EBITRatio,...,FixedAssets_w,cashholding,cashholding_w,Tangibility,Tangibility_w,ROA,ROE,ROA_w,ROE_w,_merge
0,A32,Công ty 32,UPCoM,2006,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Matched (3)
1,A32,Công ty 32,UPCoM,2007,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Matched (3)
2,A32,Công ty 32,UPCoM,2008,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Matched (3)
3,A32,Công ty 32,UPCoM,2009,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Matched (3)
4,A32,Công ty 32,UPCoM,2010,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Matched (3)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26624,YTC,XNK Y tế TP.HCM,UPCoM,2018,0.08,3.617160e+10,4.021374e+10,0.1744,0.0760,0.0683,...,4.814777e+10,0.050601,0.050601,0.062617,0.062617,0.047042,0.640009,0.047042,0.640009,Matched (3)
26625,YTC,XNK Y tế TP.HCM,UPCoM,2019,0.21,3.319102e+10,3.700330e+10,0.1836,0.0726,0.0651,...,5.160008e+10,0.065683,0.065683,0.068599,0.068599,0.044125,0.554094,0.044125,0.554094,Matched (3)
26626,YTC,XNK Y tế TP.HCM,UPCoM,2020,0.03,-1.965803e+10,-1.455882e+10,0.0861,-0.0257,-0.0347,...,5.516760e+10,0.012319,0.012319,0.055415,0.055415,-0.019746,-1.460577,-0.019746,-0.618672,Matched (3)
26627,YTC,XNK Y tế TP.HCM,UPCoM,2021,0.02,9.693024e+08,5.319168e+09,0.0868,0.0088,0.0016,...,5.406539e+10,0.017190,0.017190,0.069243,0.069243,0.001241,0.044573,0.001241,0.044573,Matched (3)


In [58]:
for i in data.columns:
    print(i)

id
Company
Index
Year
CashRatio
EBIT
EBITDA
GrossProfitMargin
EBITDARatio
EBITRatio
QuickRatio
CurrentRatio
LongDetboverOE
LongDebtoverTA
TotalDebtoverOE
TotalDebtoverTA
ShortDebtoverOE
ShortDebtoverTA
IAoverTA
TAoverOE
EBITDAoverShortDebtandExpenses
EBTMargin
NetProfitMarginRatio
TotalAssetTurnover
TotalEquityRatio
Tỷsuấtlợinhuậntừkinhdoanh
LNSTCFODoanhthu
Giátrịcổphiếu
TotalRevenue
SalesDeductions
NetRevenue
COGS
GrossProfit
RevenueFinancialActivities
FinancialExpenses
InterestExpenses
Lãilỗtừcôngtyliêndoanh
SellingExpenses
AdminExpensesCons
OperatingIncomeCons
OtherRevenue
OtherExpenses
OtherIncome
AccountingIncomePreTax
CorpTaxExpenses
CurrentCorpTax
DeferredIncomeTax
IncomeAfterDefTax
MinorityInterest
Cổđôngcủacôngtymẹ
EarningPerShare
DilutedEarningsPerShare
CURRENTASSETS
Cash
CashEquivalents
ShortTermInvestments
TradingSecurities
AllowanceDeclineInSec
HTMInvestments
CurrentReceivables
CurTradeReceivables
CurAdvancestoSuppliers
RelatedPartiesReceivables
ConstructionReceivables
Sho

In [59]:
data['ResearchDevFund_ratio']=data.ResearchDevFund/data.TotalRevenue
data['SciTechFund_ratio']=data.SciTechFund/data.TotalRevenue
# Tính ROA
data['ROA'] = data['IncomeAfterDefTax'] / data['TOTALASSETS']

# Tính ROE
data['ROE'] = data['IncomeAfterDefTax'] / data['OWNERSEQUITY']
# data['Intangible Assets']=data['CostIntanFA']/data['TOTALASSETS']
# data['Intangible Assets1']=data['AccDepIntanFA']/data['TOTALASSETS']
# data['IAoverTA']
# data['Firmsize_w']=np.log(data['TOTALASSETS'])
data['T.Debt']=data['TotalDebtoverOE']/data['TOTALASSETS']
data['T_Debt1']=data['TotalDebtoverTA']/data['TOTALASSETS']
data

,id,Company,Index,Year,CashRatio,EBIT,EBITDA,GrossProfitMargin,EBITDARatio,EBITRatio,...,Tangibility_w,ROA,ROE,ROA_w,ROE_w,_merge,ResearchDevFund_ratio,SciTechFund_ratio,T.Debt,T_Debt1
0,A32,Công ty 32,UPCoM,2006,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Matched (3),NaN,NaN,NaN,NaN
1,A32,Công ty 32,UPCoM,2007,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Matched (3),NaN,NaN,NaN,NaN
2,A32,Công ty 32,UPCoM,2008,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Matched (3),NaN,NaN,NaN,NaN
3,A32,Công ty 32,UPCoM,2009,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Matched (3),NaN,NaN,NaN,NaN
4,A32,Công ty 32,UPCoM,2010,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Matched (3),NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26624,YTC,XNK Y tế TP.HCM,UPCoM,2018,0.08,3.617160e+10,4.021374e+10,0.1744,0.0760,0.0683,...,0.062617,0.018324,0.249297,0.047042,0.640009,Matched (3),0.038055,0.0,8.037157e-12,5.852299e-13
26625,YTC,XNK Y tế TP.HCM,UPCoM,2019,0.21,3.319102e+10,3.700330e+10,0.1836,0.0726,0.0651,...,0.068599,0.020102,0.252424,0.044125,0.554094,Matched (3),0.043871,0.0,6.142000e-12,4.918918e-13
26626,YTC,XNK Y tế TP.HCM,UPCoM,2020,0.03,-1.965803e+10,-1.455882e+10,0.0861,-0.0257,-0.0347,...,0.055415,-0.012067,-0.892563,-0.019746,-0.618672,Matched (3),0.037316,0.0,1.688540e-11,2.310316e-13
26627,YTC,XNK Y tế TP.HCM,UPCoM,2021,0.02,9.693024e+08,5.319168e+09,0.0868,0.0088,0.0016,...,0.069243,0.013659,0.490421,0.001241,0.044573,Matched (3),0.000000,0.0,1.297382e-11,3.586050e-13


In [60]:
w_cols=['SciTechRatio_w','RandDRatio_w','TotalDebtoverTotalAssets_w','IAoverTA_w','TotalDebtoverTA_w','CashRatio_w','Firmsize_w','SciTechFund_w','ResearchDevFund_w','FixedAssets_w','cashholding','cashholding_w','Tangibility_w','ROA_w','ROE_w',]

In [61]:
df = data[w_cols]
df

,SciTechRatio_w,RandDRatio_w,TotalDebtoverTotalAssets_w,IAoverTA_w,TotalDebtoverTA_w,CashRatio_w,Firmsize_w,SciTechFund_w,ResearchDevFund_w,FixedAssets_w,cashholding,cashholding_w,Tangibility_w,ROA_w,ROE_w
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26624,0.0,0.026655,0.45,0.00,0.45,0.08,27.368263,0.0,2.049601e+10,4.814777e+10,0.050601,0.050601,0.062617,0.047042,0.640009
26625,0.0,0.030263,0.37,0.00,0.37,0.21,27.346266,0.0,2.276410e+10,5.160008e+10,0.065683,0.065683,0.068599,0.044125,0.554094
26626,0.0,0.022866,0.23,0.00,0.23,0.03,27.626545,0.0,2.276410e+10,5.516760e+10,0.012319,0.012319,0.055415,-0.019746,-0.618672
26627,0.0,0.000000,0.28,0.01,0.28,0.02,27.383589,0.0,0.000000e+00,5.406539e+10,0.017190,0.017190,0.069243,0.001241,0.044573


In [62]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26629 entries, 0 to 26628
Data columns (total 15 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   SciTechRatio_w              19955 non-null  float64
 1   RandDRatio_w                19944 non-null  float64
 2   TotalDebtoverTotalAssets_w  20741 non-null  float64
 3   IAoverTA_w                  20849 non-null  float64
 4   TotalDebtoverTA_w           20741 non-null  float64
 5   CashRatio_w                 20727 non-null  float64
 6   Firmsize_w                  19976 non-null  float64
 7   SciTechFund_w               19959 non-null  float64
 8   ResearchDevFund_w           19948 non-null  float64
 9   FixedAssets_w               19979 non-null  float64
 10  cashholding                 19975 non-null  float64
 11  cashholding_w               19975 non-null  float64
 12  Tangibility_w               19975 non-null  float64
 13  ROA_w                       198

In [63]:
df.describe()

,SciTechRatio_w,RandDRatio_w,TotalDebtoverTotalAssets_w,IAoverTA_w,TotalDebtoverTA_w,CashRatio_w,Firmsize_w,SciTechFund_w,ResearchDevFund_w,FixedAssets_w,cashholding,cashholding_w,Tangibility_w,ROA_w,ROE_w
count,19955.000000,19944.000000,20741.000000,20849.000000,20741.000000,20727.000000,19976.000000,1.995900e+04,1.994800e+04,1.997900e+04,19975.000000,19975.000000,19975.000000,19835.000000,19826.000000
mean,0.000204,0.044979,0.218529,0.002037,0.218529,0.577180,26.823897,2.775332e+08,4.328418e+10,4.608941e+11,0.051123,0.050167,0.271057,0.062018,0.154002
std,0.001216,0.067836,0.211974,0.009674,0.211974,1.499752,1.541666,1.791388e+09,1.285507e+11,1.440077e+12,0.067176,0.060232,0.234986,0.081134,0.201798
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,23.668734,0.000000e+00,0.000000e+00,6.097837e+06,0.000000,0.000375,0.000032,-0.212303,-0.618672
25%,0.000000,0.002381,0.010000,0.000000,0.010000,0.050000,25.766240,0.000000e+00,8.032081e+08,2.008549e+10,0.012204,0.012204,0.081791,0.016736,0.040284
50%,0.000000,0.017476,0.170000,0.000000,0.170000,0.160000,26.688649,0.000000e+00,6.512920e+09,6.552022e+10,0.030172,0.030172,0.205296,0.052580,0.133886
75%,0.000000,0.056805,0.360000,0.000000,0.360000,0.490000,27.733355,0.000000e+00,2.461194e+10,2.415835e+11,0.063946,0.063946,0.403818,0.099919,0.247876
max,0.009801,0.355467,0.860000,0.070000,0.860000,14.190000,32.382339,1.536171e+10,9.418472e+11,1.081110e+13,0.999930,0.365875,0.906662,0.332130,0.875624


In [64]:
df.drop_duplicates(inplace=True)

C:\Users\Admin\AppData\Local\Temp\ipykernel_8368\3006716147.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop_duplicates(inplace=True)


In [65]:
len(df)

19995

In [66]:
# Thay thế giá trị inf bằng NaN
df.replace([np.inf, -np.inf], np.nan, inplace=True)

# Loại bỏ các dòng chứa NaN
# data.dropna(axis=0, how='any', inplace=True)

C:\Users\Admin\AppData\Local\Temp\ipykernel_8368\3341857814.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.replace([np.inf, -np.inf], np.nan, inplace=True)


In [67]:
len(data)

26629

In [68]:
change_category = list((df.select_dtypes('object')).columns)
change_category

[]

In [69]:
df[change_category]=df[change_category].astype('category')
df.dtypes

SciTechRatio_w                float64
RandDRatio_w                  float64
TotalDebtoverTotalAssets_w    float64
IAoverTA_w                    float64
TotalDebtoverTA_w             float64
CashRatio_w                   float64
Firmsize_w                    float64
SciTechFund_w                 float64
ResearchDevFund_w             float64
FixedAssets_w                 float64
cashholding                   float64
cashholding_w                 float64
Tangibility_w                 float64
ROA_w                         float64
ROE_w                         float64
dtype: object

In [70]:
category_columns=list((df.select_dtypes('category')).columns)
numeric_columns=list((df.select_dtypes('number')).columns)

In [71]:
len(df)

19995

In [72]:
df.dropna(inplace=True)
len(df)

C:\Users\Admin\AppData\Local\Temp\ipykernel_8368\2888176961.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(inplace=True)


19777

In [73]:
df.isnull().sum()

SciTechRatio_w                0
RandDRatio_w                  0
TotalDebtoverTotalAssets_w    0
IAoverTA_w                    0
TotalDebtoverTA_w             0
CashRatio_w                   0
Firmsize_w                    0
SciTechFund_w                 0
ResearchDevFund_w             0
FixedAssets_w                 0
cashholding                   0
cashholding_w                 0
Tangibility_w                 0
ROA_w                         0
ROE_w                         0
dtype: int64

In [74]:
from sklearn.preprocessing import LabelEncoder,MinMaxScaler,OneHotEncoder,StandardScaler

label_encoders = {}
# for column in category_columns:
#     label_encoder = LabelEncoder()
#     df[column] = label_encoder.fit_transform(df[column])
#     label_encoders[column] = label_encoder  # Lưu trữ label encoder nếu cần sau này

scaler = StandardScaler()
df[numeric_columns] = scaler.fit_transform(df[numeric_columns])
df

C:\Users\Admin\AppData\Local\Temp\ipykernel_8368\1445864506.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[numeric_columns] = scaler.fit_transform(df[numeric_columns])


,SciTechRatio_w,RandDRatio_w,TotalDebtoverTotalAssets_w,IAoverTA_w,TotalDebtoverTA_w,CashRatio_w,Firmsize_w,SciTechFund_w,ResearchDevFund_w,FixedAssets_w,cashholding,cashholding_w,Tangibility_w,ROA_w,ROE_w
10,-0.167799,1.107181,-0.040588,-0.216206,-0.040588,-0.082284,0.023698,-0.155287,0.094968,-0.245513,-0.442578,-0.473473,-0.157527,-0.019337,0.088276
11,-0.167799,1.354524,0.148630,-0.216206,0.148630,-0.095759,0.071983,-0.155287,0.193513,-0.245234,0.778280,0.870570,-0.225722,-0.052296,0.049545
12,-0.167799,1.882352,-1.033981,-0.216206,-1.033981,-0.250724,0.029019,-0.155287,0.289318,-0.236631,-0.372977,-0.396849,-0.049047,0.236180,0.174626
13,-0.167799,2.591979,-0.986677,-0.216206,-0.986677,-0.203561,-0.019612,-0.155287,0.405971,-0.234714,0.788088,0.881367,0.064291,0.542404,0.256972
14,-0.167799,2.732197,-1.081286,-0.216206,-1.081286,-0.277674,0.055355,-0.155287,0.533177,-0.238064,-0.074447,-0.068197,-0.111162,0.299530,0.097699
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26624,-0.167799,-0.272446,1.047414,-0.216206,1.047414,-0.345050,0.349483,-0.155287,-0.179018,-0.287514,-0.001069,0.012584,-0.887917,-0.185185,2.405612
26625,-0.167799,-0.219182,0.668979,-0.216206,0.668979,-0.257461,0.335234,-0.155287,-0.161442,-0.285126,0.228933,0.265794,-0.862441,-0.221171,1.980102
26626,-0.167799,-0.328382,0.006716,-0.216206,0.006716,-0.378738,0.516785,-0.155287,-0.161442,-0.282658,-0.584900,-0.630155,-0.918587,-1.009352,-3.828264
26627,-0.167799,-0.665940,0.243239,0.792740,0.243239,-0.385476,0.359410,-0.155287,-0.337847,-0.283420,-0.510616,-0.548376,-0.859697,-0.750363,-0.543409


In [75]:
correl=df.corr()
correl

,SciTechRatio_w,RandDRatio_w,TotalDebtoverTotalAssets_w,IAoverTA_w,TotalDebtoverTA_w,CashRatio_w,Firmsize_w,SciTechFund_w,ResearchDevFund_w,FixedAssets_w,cashholding,cashholding_w,Tangibility_w,ROA_w,ROE_w
SciTechRatio_w,1.000000,0.075507,-0.053105,-0.007318,-0.053105,-0.004796,0.079083,0.733222,0.155480,0.106347,-0.000015,0.002116,0.031235,0.062496,0.028022
RandDRatio_w,0.075507,1.000000,-0.225131,0.039752,-0.225131,0.090741,-0.135846,0.075208,0.367011,-0.060179,0.081193,0.090672,-0.032002,0.155319,-0.010323
TotalDebtoverTotalAssets_w,-0.053105,-0.225131,1.000000,-0.027324,1.000000,-0.265724,0.273243,-0.025395,-0.020367,0.198876,-0.205330,-0.215377,0.283768,-0.144635,0.152368
IAoverTA_w,-0.007318,0.039752,-0.027324,1.000000,-0.027324,-0.011686,0.058105,-0.002127,0.037219,0.049761,0.003238,0.003462,-0.009981,-0.005253,-0.042299
TotalDebtoverTA_w,-0.053105,-0.225131,1.000000,-0.027324,1.000000,-0.265724,0.273243,-0.025395,-0.020367,0.198876,-0.205330,-0.215377,0.283768,-0.144635,0.152368
CashRatio_w,-0.004796,0.090741,-0.265724,-0.011686,-0.265724,1.000000,-0.127362,-0.005424,0.002666,-0.042977,0.181790,0.162741,-0.037068,0.057995,-0.072123
Firmsize_w,0.079083,-0.135846,0.273243,0.058105,0.273243,-0.127362,1.000000,0.225540,0.448238,0.584637,-0.198104,-0.204063,0.024658,0.004597,0.078963
SciTechFund_w,0.733222,0.075208,-0.025395,-0.002127,-0.025395,-0.005424,0.225540,1.000000,0.347348,0.296378,-0.021333,-0.021364,0.045318,0.063030,0.026454
ResearchDevFund_w,0.155480,0.367011,-0.020367,0.037219,-0.020367,0.002666,0.448238,0.347348,1.000000,0.481619,-0.032248,-0.031562,0.020072,0.089495,0.018588
FixedAssets_w,0.106347,-0.060179,0.198876,0.049761,0.198876,-0.042977,0.584637,0.296378,0.481619,1.000000,-0.082581,-0.086718,0.247184,0.019387,0.029827


In [76]:
df.columns
train=df[[
       'IAoverTA_w', 'CashRatio_w', 'Firmsize_w',
       'ResearchDevFund_w',
       'ROE_w']]

In [79]:
X = train.drop(columns=['ROE_w'])
y = train['ROE_w']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(X_train.shape,X_test.shape)

(15821, 6) (3956, 6)


In [80]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression
import pandas as pd

# # Giả sử df là DataFrame của bạn, chứa cả biến độc lập và biến mục tiêu
# # Xác định X là ma trận đặc trưng và y là vector biến mục tiêu
# X = df.drop(columns=['target_column'])
# y = df['target_column']

# Sử dụng mô hình hồi quy tuyến tính
model = LinearRegression()

# Chọn số lượng đặc trưng tối ưu bằng cách chạy RFE
# Ở đây, chúng ta chọn số lượng đặc trưng tối ưu là 5, bạn có thể thay đổi giá trị này tùy theo yêu cầu
rfe = RFE(model, n_features_to_select=5)

# Fit RFE trên dữ liệu
rfe.fit(X, y)

# In ra các đặc trưng được chọn và hạng của chúng
selected_features = pd.DataFrame({'Feature': X.columns, 'Selected': rfe.support_, 'Rank': rfe.ranking_})
print(selected_features)


             Feature  Selected  Rank
0         IAoverTA_w      True     1
1        CashRatio_w      True     1
2         Firmsize_w      True     1
3  ResearchDevFund_w     False     2
4      cashholding_w      True     1
5      Tangibility_w      True     1


In [82]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

def analyze_OLS(x,y):

    
    x = sm.add_constant(x)  # Thêm hệ số chặn vào dữ liệu

    # x.dropna(inplace=True)
    # y.dropna(inplace=True)
    # print(x.isnull().sum(),x.shape)

    # x['intercept']=const

  
    model = sm.OLS(y, x,hasconst=True)
    results = model.fit()
    summary = results.summary()
    # print(summary)
    return summary


In [83]:
print(analyze_OLS(X,y))

                            OLS Regression Results                            
Dep. Variable:                  ROE_w   R-squared:                       0.027
Model:                            OLS   Adj. R-squared:                  0.027
Method:                 Least Squares   F-statistic:                     91.83
Date:                Sun, 19 May 2024   Prob (F-statistic):          3.52e-114
Time:                        11:50:08   Log-Likelihood:                -27791.
No. Observations:               19777   AIC:                         5.560e+04
Df Residuals:                   19770   BIC:                         5.565e+04
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const              6.364e-17      0.00

In [ ]:
df.columns

Index(['id', 'Company', 'Index', 'Year', 'ROE', 'CashRatio', 'ResearchDevFund',
       'IAoverTA', 'T_Debt1', 'Firmsize_w'],
      dtype='object')

In [ ]:
# print(df.index.levels)
# 

In [ ]:
# import pandas as pd
# from linearmodels.panel import PanelOLS

# # Đọc dữ liệu từ DataFrame hoặc từ file CSV
# # df = pd.read_csv('tên_file.csv')

# # Xác định chỉ số (index) cho dữ liệu panel
# cols= ['ROE', 'CashRatio', 'ResearchDevFund',
#        'IAoverTA', 'T.Debt']
# # df_panel = df.set_index(['Company', 'Year'])
# df_panel=df[cols]

# # Chọn biến mục tiêu là ROE
# y = df_panel['ROE']

# # Chọn các biến độc lập (tất cả các cột ngoại trừ ROE)
# X = df_panel.drop(columns=['ROE'])

# # Tạo mô hình REghdfe với tùy chọn drop_absorbed=True
# mod = PanelOLS(y, X, entity_effects=False, drop_absorbed=True)

# # Fit mô hình
# res = mod.fit()

# # In kết quả
# print(res)


In [ ]:
df

,id,Company,Index,Year,ROE,CashRatio,ResearchDevFund,IAoverTA,T_Debt1,Firmsize_w
10,0,200,2,10,0.020868,-0.011624,-0.014241,-0.141903,-0.007893,0.026967
11,0,200,2,11,0.019252,-0.011831,0.014364,-0.141903,-0.007873,0.074609
12,0,200,2,12,0.020240,-0.014212,0.042173,-0.141903,-0.008094,0.032218
13,0,200,2,13,0.019905,-0.013487,0.076034,-0.141903,-0.008083,-0.015766
14,0,200,2,14,0.017265,-0.014626,0.112958,-0.141903,-0.008103,0.058203
...,...,...,...,...,...,...,...,...,...,...
26624,1534,1284,2,12,0.022772,-0.015661,-0.093771,-0.141903,-0.007844,0.348412
26625,1534,1284,2,13,0.023000,-0.014315,-0.088669,-0.141903,-0.007885,0.334352
26626,1534,1284,2,14,-0.060556,-0.016179,-0.088669,-0.141903,-0.008001,0.513485
26627,1534,1284,2,15,0.040368,-0.016282,-0.139874,0.295145,-0.007944,0.358207


In [ ]:
# import pandas as pd
# import numpy as np
# from regpyhdfe import Regpyhdfe

# # from sklearn.datasets import load_boston

# def sklearn_to_df(sklearn_dataset):
#         df = pd.DataFrame(sklearn_dataset.data, columns=sklearn_dataset.feature_names)
#         df['target'] = pd.Series(sklearn_dataset.target)
#         return df

# # df = sklearn_to_df(load_boston())

# # df.to_stata("boston.dta")
# # . reghdfe target CRIM ZN INDUS NOX AGE, absorb(CHAS RAD)
# # (MWFE estimator converged in 3 iterations)
# #
# # HDFE Linear regression                            Number of obs   =        506
# # Absorbing 2 HDFE groups                           F(   5,    491) =      21.93
# #                                                   Prob > F        =     0.0000
# #                                                   R-squared       =     0.3887
# #                                                   Adj R-squared   =     0.3712
# #                                                   Within R-sq.    =     0.1825
# #                                                   Root MSE        =     7.2929
# #
# # ------------------------------------------------------------------------------
# #       target |      Coef.   Std. Err.      t    P>|t|     [95% Conf. Interval]
# # -------------+----------------------------------------------------------------
# #         CRIM |  -.2089114   .0491012    -4.25   0.000    -.3053857   -.1124371
# #           ZN |   .0679261   .0183051     3.71   0.000       .03196    .1038922
# #        INDUS |  -.2279553   .0860909    -2.65   0.008    -.3971074   -.0588033
# #          NOX |  -9.424849   5.556005    -1.70   0.090    -20.34133     1.49163
# #          AGE |  -.0140739   .0183467    -0.77   0.443    -.0501215    .0219738
# #        _cons |   31.24755    2.53596    12.32   0.000     26.26487    36.23022
# # ------------------------------------------------------------------------------
# #
# # Absorbed degrees of freedom:
# # -----------------------------------------------------+
# #  Absorbed FE | Categories  - Redundant  = Num. Coefs |
# # -------------+---------------------------------------|
# #         CHAS |         2           0           2     |
# #          RAD |         9           1           8     |
# # -----------------------------------------------------+

# # target~CRIM + ZN + INDUS + NOX + AGE, absorb(CHAS, RAD)
# #                                  OLS Regression Results
# # =======================================================================================
# # Dep. Variable:                 target   R-squared (uncentered):                   0.183
# # Model:                            OLS   Adj. R-squared (uncentered):              0.158
# # Method:                 Least Squares   F-statistic:                              21.93
# # Date:                Mon, 11 Jan 2021   Prob (F-statistic):                    7.57e-20
# # Time:                        20:30:53   Log-Likelihood:                         -1715.7
# # No. Observations:                 506   AIC:                                      3441.
# # Df Residuals:                     491   BIC:                                      3463.
# # Df Model:                           5
# # Covariance Type:            nonrobust
# # ==============================================================================
# #                  coef    std err          t      P>|t|      [0.025      0.975]
# # ------------------------------------------------------------------------------
# # CRIM          -0.2089      0.049     -4.255      0.000      -0.305      -0.112
# # ZN             0.0679      0.018      3.711      0.000       0.032       0.104
# # INDUS         -0.2280      0.086     -2.648      0.008      -0.397      -0.059
# # NOX           -9.4248      5.556     -1.696      0.090     -20.341       1.492
# # AGE           -0.0141      0.018     -0.767      0.443      -0.050       0.022
# # ==============================================================================
# # Omnibus:                      172.457   Durbin-Watson:                   0.904
# # Prob(Omnibus):                  0.000   Jarque-Bera (JB):              532.297
# # Skew:                           1.621   Prob(JB):                    2.59e-116
# # Kurtosis:                       6.839   Cond. No.                         480.
# # ==============================================================================
# #
# # Notes:
# # [1] R² is computed without centering (uncentered) since the model does not contain a constant.
# # [2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
# model = Regpyhdfe(df, 'ROE', ['CashRatio', 'ResearchDevFund',
#        'IAoverTA', 'T_Debt1','Firmsize_w'],['id','Year','Index'])
# results = model.fit()
# # print("target~CRIM + ZN + INDUS + NOX 
# # + AGE, absorb(CHAS, RAD)")
# print(results.summary())

> c:\users\pc\miniconda3\envs\uii_prj\lib\site-packages\regpyhdfe\regpyhdfe.py(114)fit()
    112                 a = 1234
    113                 import pdb; pdb.set_trace()
--> 114                 self.model.df_resid = np.sum(~self.algo._singleton_indices)-len(self.predictors)-self.algo.degrees
    115             return self.model.fit()
    116 

*** NameError: name 'ipdb' is not defined


In [ ]:
import pandas as pd
import numpy as np
from regpyhdfe import Regpyhdfe

# from sklearn.datasets import load_boston

def sklearn_to_df(sklearn_dataset):
        df = pd.DataFrame(sklearn_dataset.data, columns=sklearn_dataset.feature_names)
        df['target'] = pd.Series(sklearn_dataset.target)
        return df

# df = sklearn_to_df(load_boston())

# df.to_stata("boston.dta")
# . reghdfe target CRIM ZN INDUS NOX AGE, absorb(CHAS RAD)
# (MWFE estimator converged in 3 iterations)
#
# HDFE Linear regression                            Number of obs   =        506
# Absorbing 2 HDFE groups                           F(   5,    491) =      21.93
#                                                   Prob > F        =     0.0000
#                                                   R-squared       =     0.3887
#                                                   Adj R-squared   =     0.3712
#                                                   Within R-sq.    =     0.1825
#                                                   Root MSE        =     7.2929
#
# ------------------------------------------------------------------------------
#       target |      Coef.   Std. Err.      t    P>|t|     [95% Conf. Interval]
# -------------+----------------------------------------------------------------
#         CRIM |  -.2089114   .0491012    -4.25   0.000    -.3053857   -.1124371
#           ZN |   .0679261   .0183051     3.71   0.000       .03196    .1038922
#        INDUS |  -.2279553   .0860909    -2.65   0.008    -.3971074   -.0588033
#          NOX |  -9.424849   5.556005    -1.70   0.090    -20.34133     1.49163
#          AGE |  -.0140739   .0183467    -0.77   0.443    -.0501215    .0219738
#        _cons |   31.24755    2.53596    12.32   0.000     26.26487    36.23022
# ------------------------------------------------------------------------------
#
# Absorbed degrees of freedom:
# -----------------------------------------------------+
#  Absorbed FE | Categories  - Redundant  = Num. Coefs |
# -------------+---------------------------------------|
#         CHAS |         2           0           2     |
#          RAD |         9           1           8     |
# -----------------------------------------------------+

# target~CRIM + ZN + INDUS + NOX + AGE, absorb(CHAS, RAD)
#                                  OLS Regression Results
# =======================================================================================
# Dep. Variable:                 target   R-squared (uncentered):                   0.183
# Model:                            OLS   Adj. R-squared (uncentered):              0.158
# Method:                 Least Squares   F-statistic:                              21.93
# Date:                Mon, 11 Jan 2021   Prob (F-statistic):                    7.57e-20
# Time:                        20:30:53   Log-Likelihood:                         -1715.7
# No. Observations:                 506   AIC:                                      3441.
# Df Residuals:                     491   BIC:                                      3463.
# Df Model:                           5
# Covariance Type:            nonrobust
# ==============================================================================
#                  coef    std err          t      P>|t|      [0.025      0.975]
# ------------------------------------------------------------------------------
# CRIM          -0.2089      0.049     -4.255      0.000      -0.305      -0.112
# ZN             0.0679      0.018      3.711      0.000       0.032       0.104
# INDUS         -0.2280      0.086     -2.648      0.008      -0.397      -0.059
# NOX           -9.4248      5.556     -1.696      0.090     -20.341       1.492
# AGE           -0.0141      0.018     -0.767      0.443      -0.050       0.022
# ==============================================================================
# Omnibus:                      172.457   Durbin-Watson:                   0.904
# Prob(Omnibus):                  0.000   Jarque-Bera (JB):              532.297
# Skew:                           1.621   Prob(JB):                    2.59e-116
# Kurtosis:                       6.839   Cond. No.                         480.
# ==============================================================================
#
# Notes:
# [1] R² is computed without centering (uncentered) since the model does not contain a constant.
# [2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
model = Regpyhdfe(df, 'ROE', ['CashRatio', 'ResearchDevFund',
       'IAoverTA', 'T_Debt1','Firmsize_w'],['id','Year','Index'])
results = model.fit()
# print("target~CRIM + ZN + INDUS + NOX 
# + AGE, absorb(CHAS, RAD)")
print(results.summary())